# Generate Reports

In [1]:
import pandas as pd
import asyncio
import pathlib
import json
import traceback
import stamina


from kruppe.llm import OpenAILLM 

In [2]:
llm = OpenAILLM(model="gpt-4.1-mini") # using a single llm client


## Kruppe

### Initialization

In [3]:
# logging imports
import logging
from logging import StreamHandler

# toolkit import
from kruppe.llm import OpenAIEmbeddingModel
from kruppe.functional.docstore.mongo_store import MongoDBStore
from kruppe.functional.rag.vectorstore.chroma import ChromaVectorStore
from kruppe.functional.rag.index.vectorstore_index import VectorStoreIndex
from kruppe.functional.rag.retriever.simple_retriever import SimpleRetriever
from kruppe.functional.rag.retriever.fusion_retriever import QueryFusionRetriever
from kruppe.functional.ragquery import RagQuery
from kruppe.functional.llmquery import LLMQuery
from kruppe.functional.newshub import NewsHub
from kruppe.functional.finhub import FinHub
from kruppe.data_source.news.nyt import NewYorkTimesData
from kruppe.data_source.news.ft import FinancialTimesData
from kruppe.data_source.news.newsapi import NewsAPIData
from kruppe.data_source.finance.yfin import YFinanceData

# researcher import
from kruppe.algorithm.librarian import Librarian
from kruppe.algorithm.coordinator import Coordinator

In [4]:
# set up logging

# handlers
formatter = logging.Formatter('%(asctime)s - %(pathname)s - %(levelname)s - %(message)s')
short_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# set up logging for jupyter notebook
ch = StreamHandler()
ch.setFormatter(formatter)
ch.setLevel(logging.INFO)

# set up logging for everything
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/everything.log"
with open(log_file_path, 'w') as f:
    pass

fh_all = logging.FileHandler(log_file_path)
fh_all.setFormatter(formatter)
fh_all.setLevel(logging.DEBUG)

log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/kruppe.log"
with open(log_file_path, 'w') as f:
    pass
fh_kruppe = logging.FileHandler(log_file_path)
fh_kruppe.setFormatter(short_formatter)
fh_kruppe.setLevel(logging.DEBUG)

# set up loggers
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
root_logger.handlers.clear()
root_logger.addHandler(fh_all) # log everything to a file

kruppe_logger = logging.getLogger("kruppe")
kruppe_logger.setLevel(logging.INFO)
kruppe_logger.handlers.clear()
kruppe_logger.addHandler(fh_kruppe) # log everything to a file
kruppe_logger.propagate = False

algo_logger = logging.getLogger("kruppe.algorithm")
algo_logger.setLevel(logging.INFO)
algo_logger.addHandler(ch) # log to console


In [5]:
reset_db=False

# db_name = "kruppe_librarian"
# collection_name = "general_news_04_20_2025"
db_name = "kruppe_librarian_3"
collection_name = "cloudy_leopard"

# Create doc store
unique_indices = [['title', 'datasource']] # NOTE: this is important to avoid duplicates
docstore = await MongoDBStore.acreate_db(
    db_name=db_name,
    collection_name=collection_name,
    unique_indices=unique_indices,
    reset_db=reset_db
)

# Create vectorstore index
embedding_model = OpenAIEmbeddingModel()
vectorstore = ChromaVectorStore(
    embedding_model=embedding_model,
    collection_name=collection_name,
    persist_path=f'/Volumes/Lexar/Daniel Liu/vectorstores/{db_name}'
)

index = VectorStoreIndex(vectorstore=vectorstore)
simple_retriever = SimpleRetriever(index=index)
retriever = QueryFusionRetriever(
    retrievers=[simple_retriever],
    mode='rrf',
    llm=llm,
    num_queries=3
)


if reset_db:
    vectorstore.clear()


# docs = await docstore.aget_all_documents()
# print(len(docs))
# await index.async_add_documents(docs)

print("Number of documents:", docstore.size())
print("Number of chunks:", vectorstore.size())


Number of documents: 0
Number of chunks: 0


In [6]:
rag_query_engine = RagQuery(
    retriever = retriever,
    llm = llm
)

llm_query_engine = LLMQuery(
    llm = llm
)

news_hub = NewsHub(news_sources=[
    NewYorkTimesData(headers_path="/Users/danielliu/Workspace/fin-rag/.nyt-headers.json"),
    FinancialTimesData(headers_path="/Users/danielliu/Workspace/fin-rag/.ft-headers.json"),
    NewsAPIData()
])

fin_hub = FinHub(
    fin_source = YFinanceData(),
    llm = llm
)

In [7]:
toolkit_librarian = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    news_hub.news_recent,
    news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

toolkit_researcher = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    # news_hub.news_recent,
    # news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

In [8]:
librarian = Librarian(
    llm=llm,
    toolkit=toolkit_librarian,
    docstore=docstore,
    index=index,
    max_steps=20,
    verbose=False
)

In [9]:
tree_configs = {
    "llm": llm,
    "toolkit": toolkit_researcher,
    "docstore": docstore,
    "index": index,
    "max_step": 15,
    "max_degree": 2,
    "verbose": False
}

In [10]:
n_experts = 3

In [11]:
questions = [
    "How might Lockheed Martin’s stock performance be affected in the context of U.S.-Ukraine mining agreements?",
    "What impact will the long-term deployment of U.S. B-1B bombers in Japan have on the situation on the Korean Peninsula, and how could this influence Lockheed Martin’s stock performance?",
    "What are the potential long-term implications for Lockheed Martin if the U.S. Department of Defense explicitly excludes the company from participation in the sixth-generation fighter jet program?",
    "Given the continued decline in shale oil extraction costs in the United States in recent years, how significantly might this affect Energy Transfer’s operations? What could be the resulting impact on its stock price and dividend payouts?",
    "What would be the economic impact on the United States if China retaliates against tariffs imposed by Donald Trump? How might this affect various sectors of the U.S. stock market, and what implications could it have for Federal Reserve policy?"
]

In [12]:
import chromadb
import chromadb.errors

failed = False

semaphore = asyncio.Semaphore(1)  # Limit to n concurrent tasks

async def execute_with_semaphore(question: str, semaphore: asyncio.Semaphore):
    global failed
    global llm

    if failed:
        print(f"Question {question} has failed due to some global error. Skipping.")
        return []
    
    if llm.price() > 200:
        print("LLM has exceeded the budget. Ending the experiment.")
        return []
    
    # 3 retries
    async with semaphore:
        coordinator = Coordinator(
            llm=llm,
            tree_configs = tree_configs,
            librarian = librarian,
        )

        try:
            
            for attempt in range(3):
                print(f"Attempt {attempt} to answer question: {question}")
                try:
                    reports = await coordinator.execute(query=question, n_experts=n_experts)
                    break  # Break if successful
                except* chromadb.errors.InternalError as e:
                    
                    if attempt == 2:
                        raise # re-raise the last exception
                    
                    # TODO
                    failed = True
                    print(f"Attempt {attempt + 1} failed with Chroma DB InternalError: {e}")
                    raise
                    # print("Resetting vectorstore and retrying...")
                    # reset_vectorstore()
                    
            
        except* Exception as eg:  # catch *all* sub-exceptions
            print(f"Error when generating report for question: {question}")
            print("Caught ExceptionGroup:")
            for sub in eg.exceptions:
                print("--- sub-exception:")
                traceback.print_exception(type(sub), sub, sub.__traceback__)

            if coordinator.research_reports:
                print("Partial report generated. Returning the partial reports.")
            else:
                print("No report generated.")
    

        
        print(f"Research completed for question: {question}")
        print(f"LLM current cost: {llm.price()}")

        return coordinator.research_reports
    
    return []

async with asyncio.TaskGroup() as tg:
    tasks = []
    for question in questions:
        task = tg.create_task(execute_with_semaphore(question, semaphore))
        tasks.append(task)
    
results = [task.result() for task in tasks]
print("All tasks completed.")

Attempt 0 to answer question: How might Lockheed Martin’s stock performance be affected in the context of U.S.-Ukraine mining agreements?
Background report generated for query: How might Lockheed Martin’s stock performance be affected in the context of U.S.-Ukraine mining agreements?
Domain experts generated: 3 experts found from 10 generated.


2025-05-01 12:20:58,937 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction, action
2025-05-01 12:20:59,082 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction, action
2025-05-01 12:21:24,836 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction, action
2025-05-01 12:21:48,874 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction
2025-05-01 12:22:58,253 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return

Researcher 7657ac26-a895-4eb7-b4e0-8c45cdcf7634 completed with 2 reports.
Researcher 9786c6ca-ab76-442a-8c64-6693c726ed58 completed with 2 reports.
Researcher 9c244694-ae0e-42df-8adb-7226a42fee82 completed with 2 reports.
Completed 6 research reports for query: How might Lockheed Martin’s stock performance be affected in the context of U.S.-Ukraine mining agreements?.
Research completed for question: How might Lockheed Martin’s stock performance be affected in the context of U.S.-Ukraine mining agreements?
LLM current cost: 0.5350124
Attempt 0 to answer question: What impact will the long-term deployment of U.S. B-1B bombers in Japan have on the situation on the Korean Peninsula, and how could this influence Lockheed Martin’s stock performance?
Background report generated for query: What impact will the long-term deployment of U.S. B-1B bombers in Japan have on the situation on the Korean Peninsula, and how could this influence Lockheed Martin’s stock performance?
Domain experts genera

2025-05-01 12:30:47,526 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis


Researcher c3034b6f-f3f6-414d-90ce-31ef267f6d0d completed with 2 reports.
Researcher 0c5d180b-f316-4c56-aacc-cf6f551e1cab completed with 2 reports.
Researcher 0efa21a7-0d8f-4d2d-a9fa-e02267f92049 completed with 2 reports.
Completed 6 research reports for query: What impact will the long-term deployment of U.S. B-1B bombers in Japan have on the situation on the Korean Peninsula, and how could this influence Lockheed Martin’s stock performance?.
Research completed for question: What impact will the long-term deployment of U.S. B-1B bombers in Japan have on the situation on the Korean Peninsula, and how could this influence Lockheed Martin’s stock performance?
LLM current cost: 1.0549708
Attempt 0 to answer question: What are the potential long-term implications for Lockheed Martin if the U.S. Department of Defense explicitly excludes the company from participation in the sixth-generation fighter jet program?
Background report generated for query: What are the potential long-term implicat

...

Researcher f98fc53f-b44a-4946-b834-f99e243c6910 completed with 2 reports.


2025-05-01 12:55:19,898 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction


Researcher 33765e73-8ec8-4251-b358-c3af5e1dfe50 completed with 2 reports.
Researcher 9f6e2702-dbc2-4b24-abe5-dbd0a07e87e7 completed with 2 reports.
Completed 6 research reports for query: Given the continued decline in shale oil extraction costs in the United States in recent years, how significantly might this affect Energy Transfer’s operations? What could be the resulting impact on its stock price and dividend payouts?.
Research completed for question: Given the continued decline in shale oil extraction costs in the United States in recent years, how significantly might this affect Energy Transfer’s operations? What could be the resulting impact on its stock price and dividend payouts?
LLM current cost: 2.707644
Attempt 0 to answer question: What would be the economic impact on the United States if China retaliates against tariffs imposed by Donald Trump? How might this affect various sectors of the U.S. stock market, and what implications could it have for Federal Reserve policy?
B

2025-05-01 13:00:25,420 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis
2025-05-01 13:01:38,090 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction
2025-05-01 13:02:35,542 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction


Researcher a9d62a22-15b5-4c23-9e6a-9b07e9db136d completed with 2 reports.
Researcher 40e001a8-020d-430f-bef1-21347e8a86c8 completed with 2 reports.
Researcher aade7abb-445f-40a1-8ca2-36c6374a0b7c completed with 2 reports.
Completed 6 research reports for query: What would be the economic impact on the United States if China retaliates against tariffs imposed by Donald Trump? How might this affect various sectors of the U.S. stock market, and what implications could it have for Federal Reserve policy?.
Research completed for question: What would be the economic impact on the United States if China retaliates against tariffs imposed by Donald Trump? How might this affect various sectors of the U.S. stock market, and what implications could it have for Federal Reserve policy?
LLM current cost: 3.2584144000000004
All tasks completed.


In [17]:
for i, question in enumerate(questions):
    print(f"Question {i + 1}: {question}")
    if results[i]:
        for report in results[i]:
            print("Expert Perspective:", report.metadata['expert'])
            print("Expert Description:", report.metadata['expert_description'])
            print("Report:")
            print(report.text)
            print('--'*20)
    else:
        print("No report generated.")
    print("=="*20)

Question 1: How might Lockheed Martin’s stock performance be affected in the context of U.S.-Ukraine mining agreements?
Expert Perspective: Critical Minerals and Mining Analyst
Expert Description: The Critical Minerals and Mining Analyst has expertise in the global supply chain, extraction, and trade of strategic minerals essential for defense technology manufacturing, evaluating how new resource agreements shift market dynamics and company cost structures.
Report:
Final Hypothesis:
The U.S.-Ukraine mining agreements offer Lockheed Martin potential medium- to long-term benefits by diversifying and securing access to critical minerals that are currently dominated by geopolitically risky suppliers like China. This enhanced supply chain security can improve operational stability and cost predictability, ultimately supporting positive stock performance. However, near-term stock impact is likely to be limited due to significant execution and geopolitical risks in Ukraine’s mining sector and

In [16]:
data = []
for question, q_results in zip(questions, results):
    reports = [report.model_dump(mode='json') for report in q_results]
    data.append({
        "question": question,
        "research_reports": reports
    })

with open("./shuic_reports.json", "w") as f:
    json.dump(data, f, indent=4)